<a href="https://colab.research.google.com/github/AlySerry0/CSEN903-Airline-Team-133/blob/main/Airline_133.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATA FETCHING

In [ ]:
!git clone https://github.com/AlySerry0/CSEN903-Airline-RawData.git

raw_data_dir = '/content/CSEN903-Airline-RawData'

In [ ]:
!pip install vaderSentiment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [219]:
AirlineScrappedReview = pd.read_csv(f'{raw_data_dir}/AirlineScrappedReview_Cleaned.csv')
Customer_comment = pd.read_csv(f'{raw_data_dir}/Customer_comment.csv')
Passanger_Booking_Data = pd.read_csv(f'{raw_data_dir}/Passanger_booking_data.csv')
Survey_Data_Inflight_Satisfaction_Score = pd.read_csv(f'{raw_data_dir}/Survey data_Inflight Satisfaction Score.csv')

DATA CLEANING

In [ ]:
AirlineScrappedReview.info()

In [ ]:
AirlineScrappedReview.head()

In [ ]:
AirlineScrappedReview.isnull().sum()

In [ ]:
AirlineScrappedReview.describe(include='all')

In [220]:
columns_to_drop = [
    'Flying_Date',        # Too many missing values
    'Passanger_Name', 'Review_title',     # Unneeded identifiers
]

AirlineScrappedReview['Layover_Route'] = AirlineScrappedReview['Layover_Route'].fillna("None")
AirlineScrappedReview = AirlineScrappedReview.drop(columns=columns_to_drop)
AirlineScrappedReview = AirlineScrappedReview.replace('Unknown', np.nan)
AirlineScrappedReview = AirlineScrappedReview.dropna()
AirlineScrappedReview = AirlineScrappedReview.drop_duplicates()

In [ ]:
# Display the final prepped DataFrame
print("\n--- Final Prepped DataFrame ---")
AirlineScrappedReview.head()

In [ ]:
Customer_comment.info()

In [ ]:
Customer_comment.head()

In [ ]:
Customer_comment.isnull().sum()

In [ ]:
Customer_comment.describe(include='all')

In [ ]:
print("\n--- Unique Values in Key Columns ---")
cols_to_check = ['arrival_delay_group', 'departure_delay_group', 'entity',
                 'seat_factor_band', 'loyalty_program_level', 'fleet_usage', 'response_group', 'sentiments']

for col in cols_to_check:
    print(f"Column '{col}': {Customer_comment[col].unique()}")

In [ ]:
# --- 1. Drop Unnecessary/Redundant Columns ---
columns_to_drop = [
    'Unnamed: 0',         # Duplicate index
    'ques_verbatim_text', # Only has one unique value, no info
    'transformed_text'    # Incomplete, and we have the raw 'verbatim_text'
]
Customer_comment = Customer_comment.drop(columns=columns_to_drop)

# --- 2. Handle Missing Data ---
# Fill missing loyalty levels with 'None'
Customer_comment['loyalty_program_level'] = Customer_comment['loyalty_program_level'].fillna('None')

# --- 3. Convert Data Types ---
# Convert date column to datetime objects
Customer_comment['scheduled_departure_date'] = pd.to_datetime(Customer_comment['scheduled_departure_date'])

Customer_comment = Customer_comment.drop_duplicates()

Customer_comment.info()
Customer_comment.head()

In [ ]:
Passanger_Booking_Data.info()

In [ ]:
Passanger_Booking_Data.head()

In [ ]:
Passanger_Booking_Data.isnull().sum()

In [ ]:
Passanger_Booking_Data.describe(include='all')

In [ ]:
print(Survey_Data_Inflight_Satisfaction_Score['score'].unique())
print(Survey_Data_Inflight_Satisfaction_Score['question_text'].unique())
print(Survey_Data_Inflight_Satisfaction_Score['driver_sub_group1'].unique())
print(Survey_Data_Inflight_Satisfaction_Score['driver_sub_group2'].unique())
print(Survey_Data_Inflight_Satisfaction_Score['media_provider'].unique())

Survey_Data_Inflight_Satisfaction_Score.info()

In [ ]:
is_score = Survey_Data_Inflight_Satisfaction_Score['score'].str.isnumeric()
Survey_Data_Inflight_Satisfaction_Score['satisfaction_score'] = np.where(
    is_score,
    Survey_Data_Inflight_Satisfaction_Score['score'],
    np.nan
)

Survey_Data_Inflight_Satisfaction_Score['food_item_chosen'] = np.where(
    ~is_score,
    Survey_Data_Inflight_Satisfaction_Score['score'],
    np.nan
)


Survey_Data_Inflight_Satisfaction_Score['satisfaction_score'] = pd.to_numeric(Survey_Data_Inflight_Satisfaction_Score['satisfaction_score'], errors='coerce').astype('Int64')


Survey_Data_Inflight_Satisfaction_Score = Survey_Data_Inflight_Satisfaction_Score.drop('score', axis=1)


print(Survey_Data_Inflight_Satisfaction_Score[['satisfaction_score', 'food_item_chosen', 'question_text']].head(10))

Survey_Data_Inflight_Satisfaction_Score = Survey_Data_Inflight_Satisfaction_Score.drop(columns=['cabin_name'])
Survey_Data_Inflight_Satisfaction_Score['loyalty_program_level'] = Survey_Data_Inflight_Satisfaction_Score['loyalty_program_level'].fillna('None')
Survey_Data_Inflight_Satisfaction_Score = Survey_Data_Inflight_Satisfaction_Score.dropna(subset=['departure_gate'])
Survey_Data_Inflight_Satisfaction_Score = Survey_Data_Inflight_Satisfaction_Score.dropna(subset=['arrival_gate'])
Survey_Data_Inflight_Satisfaction_Score['media_provider'] = Survey_Data_Inflight_Satisfaction_Score['media_provider'].fillna(Survey_Data_Inflight_Satisfaction_Score['media_provider'].mode()[0])


In [ ]:
Survey_Data_Inflight_Satisfaction_Score.head()

In [ ]:
Survey_Data_Inflight_Satisfaction_Score.isnull().sum()

In [ ]:
Survey_Data_Inflight_Satisfaction_Score.describe(include='all')

DATA ENGINEERING QUESTIONS

In [221]:
AirlineScrappedReview['Satisfaction'] = (AirlineScrappedReview['Rating'] >= 5).astype(int)
sia = SentimentIntensityAnalyzer()
AirlineScrappedReview['Sentiment_Score'] = AirlineScrappedReview['Review_content'].apply(
    lambda review: sia.polarity_scores(review)['compound']
)

In [ ]:
top_10_routes = AirlineScrappedReview['Route'].value_counts().head(10)
print("The most popular 10 routes are:" , top_10_routes)

In [ ]:
bookings_df = Passanger_Booking_Data

# 2. Calculate the number of completed bookings for each flight hour
hourly_bookings = bookings_df['flight_hour'].value_counts().sort_index()

plt.figure(figsize=(14, 6))

# Create a bar chart
hourly_bookings.plot(kind='bar', color='darkred') # Changed color for distinction

plt.title('Distribution of Bookings Across Flight Hours', fontsize=16)
plt.xlabel('Flight Hour (0 = 12 AM, 23 = 11 PM)', fontsize=12)
plt.ylabel('Number of Bookings', fontsize=12)
plt.xticks(rotation=0, ha='center') # Ensure hours are readable
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save the plot with a new name
plt.savefig('flight_hour_distribution.png')

In [ ]:
completed_bookings_df = Passanger_Booking_Data[Passanger_Booking_Data['booking_complete'] == 1]

# 2. Calculate the number of completed bookings for each flight hour
hourly_bookings = completed_bookings_df['flight_hour'].value_counts().sort_index()

plt.figure(figsize=(14, 6))

# Create a bar chart
hourly_bookings.plot(kind='bar', color='darkgreen') # Changed color for distinction

plt.title('Distribution of COMPLETED Bookings Across Flight Hours', fontsize=16)
plt.xlabel('Flight Hour (0 = 12 AM, 23 = 11 PM)', fontsize=12)
plt.ylabel('Number of Completed Bookings', fontsize=12)
plt.xticks(rotation=0, ha='center') # Ensure hours are readable
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save the plot with a new name
plt.savefig('completed_flight_hour_distribution.png')

In [ ]:
combination_ratings = AirlineScrappedReview.groupby(['Traveller_Type', 'Class'])['Rating'].mean()

highest_rating_combo = combination_ratings.idxmax()
highest_rating_score = combination_ratings.max()

lowest_rating_combo = combination_ratings.idxmin()
lowest_rating_score = combination_ratings.min()

print("--- Traveler Type and Class Rating Analysis ---")
print("\n🏆 Highest Rated Combination:")
print(f"  Combination: {highest_rating_combo[0]} in {highest_rating_combo[1]}")
print(f"  Average Rating: {highest_rating_score:.3f}")

print("\n📉 Lowest Rated Combination:")
print(f"  Combination: {lowest_rating_combo[0]} in {lowest_rating_combo[1]}")
print(f"  Average Rating: {lowest_rating_score:.3f}")

ENCODING

In [ ]:
AirlineScrappedReview['Verified'] = AirlineScrappedReview['Verified'].map({
    'Trip Verified': 1,
    'Not Verified': 0
})

AirlineScrappedReview = pd.get_dummies(AirlineScrappedReview,
                            columns=['Traveller_Type', 'Class'],
                            drop_first=True)

AirlineScrappedReview['is_layover'] = (AirlineScrappedReview['Layover_Route'] != 'None').astype(int)

def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371
    return c * r

AirlineScrappedReview['flight_distance_km'] = haversine(
    AirlineScrappedReview['Start_Latitude'],
    AirlineScrappedReview['Start_Longitude'],
    AirlineScrappedReview['End_Latitude'],
    AirlineScrappedReview['End_Longitude']
)

start_freq = AirlineScrappedReview['Start_Location'].map(AirlineScrappedReview['Start_Location'].value_counts())
end_freq = AirlineScrappedReview['End_Location'].map(AirlineScrappedReview['End_Location'].value_counts())
AirlineScrappedReview['start_loc_freq'] = start_freq
AirlineScrappedReview['end_loc_freq'] = end_freq

AirlineScrappedReview['start_country'] = AirlineScrappedReview['Start_Address'].apply(lambda x: x.split(',')[-1].strip())
AirlineScrappedReview['end_country'] = AirlineScrappedReview['End_Address'].apply(lambda x: x.split(',')[-1].strip())

AirlineScrappedReview['is_international'] = (AirlineScrappedReview['start_country'] != AirlineScrappedReview['end_country']).astype(int)

AirlineScrappedReview = AirlineScrappedReview.drop(['Start_Address', 'End_Address', 'Layover_Route', 'Start_Latitude', 'Start_Longitude', 'End_Latitude', 'End_Longitude', 'start_country', 'end_country'], axis=1)

AirlineScrappedReview.head()

In [ ]:
# --- 4. Encode Binary Categorical Columns ---
# We'll map the "negative" or "delayed" state to 1 and the "positive" state to 0

Customer_comment['arrival_delay_group'] = Customer_comment['arrival_delay_group'].map({
    'Delayed': 1,
    'Early & Ontime': 0
})

Customer_comment['departure_delay_group'] = Customer_comment['departure_delay_group'].map({
    'Delayed': 1,
    'Early & Ontime': 0
})

Customer_comment['sentiments'] = Customer_comment['sentiments'].map({
    'Negative': 1,
    'Neutral': 0
})

# --- 5. Encode Ordinal 'seat_factor_band' Column ---
band_mapping = {
    '0 to 70': 0,
    '70+': 1,
    '80+': 2,
    '90+': 3
}

Customer_comment['seat_factor_band'] = Customer_comment['seat_factor_band'].map(band_mapping)